In [30]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import chi2_contingency

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import LabelPowerset #bu komple sonradan

from catboost import CatBoostClassifier, Pool #pool sonradan

import warnings
warnings.filterwarnings('ignore')

In [31]:
import tensorflow as tf
print(tf.test.is_gpu_available())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

True
Num GPUs Available:  1


In [32]:
from fastparquet import ParquetFile

# Parquet File okuma
pf = ParquetFile("train_final.parquet")
pf_test = ParquetFile("test_final.parquet")
pf_sub = ParquetFile("submission_sample_final.parquet")
# Bir pandas dataFrame'ine dönüştürme
df_train = pf.to_pandas()
df_test = pf_test.to_pandas()
df_sub = pf_sub.to_pandas()

#Csv olarak kaydetme
#dataFrame.to_csv("train_final.csv", index = False)

In [33]:
df_train['corr_48_49'] = df_train['feature_48'] + df_train['feature_49']
df_test['corr_48_49'] = df_test['feature_48'] + df_test['feature_49']

# Min-Max ölçeklendirme nesnesini oluşturun
scaler48 = MinMaxScaler(feature_range=(-5, 5))
scaler49 = MinMaxScaler(feature_range=(-5, 5))
scalercorr = MinMaxScaler(feature_range=(-5, 5))
#scaler_log = MinMaxScaler(feature_range=(-5, 5))

df_train['feature_48'] = scaler48.fit_transform(df_train[['feature_48']])
df_train['feature_49'] = scaler49.fit_transform(df_train[['feature_49']])
df_train['corr_48_49'] = scalercorr.fit_transform(df_train[['corr_48_49']])
#df_train['log+1'] = scaler_log.fit_transform(df_train[['log+1']])


df_test['feature_48'] = scaler48.transform(df_test[['feature_48']])
df_test['feature_49'] = scaler49.transform(df_test[['feature_49']])
df_test['corr_48_49'] = scalercorr.transform(df_test[['corr_48_49']])
#df_test['log+1'] = scaler_log.transform(df_test[['log+1']])

In [34]:
carrier_counts = df_train['carrier'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
low_carrier_mask = carrier_counts <= 400#400-20
low_carriers = carrier_counts[low_carrier_mask].index.tolist()
df_train.loc[df_train['carrier'].isin(low_carriers), 'carrier'] = 'other'

carrier_counts = df_test['carrier'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
low_carrier_mask = carrier_counts < 50#50-3
low_carriers = carrier_counts[low_carrier_mask].index.tolist()
df_test.loc[df_test['carrier'].isin(low_carriers), 'carrier'] = 'other'

In [35]:
df_train['carrier'] = df_train['carrier'].str.replace(' ', '_')
#df_train['carrier'] = df_train['carrier'].str.replace('TR', '')

df_test['carrier'] = df_test['carrier'].str.replace(' ', '_')
#df_test['carrier'] = df_test['carrier'].str.replace('TR', '')

In [36]:
devicebrand_counts = df_train['devicebrand'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
devicebrand_mask = devicebrand_counts <= 400#400
devicebrand = devicebrand_counts[devicebrand_mask].index.tolist()
df_train.loc[df_train['devicebrand'].isin(devicebrand), 'devicebrand'] = 'other'

devicebrand_counts = df_test['devicebrand'].value_counts()
# Frekansı 400'den küçük olanları "low" olarak işaretleyin
devicebrand_mask = devicebrand_counts < 50#50
devicebrand = devicebrand_counts[devicebrand_mask].index.tolist()
df_test.loc[df_test['devicebrand'].isin(devicebrand), 'devicebrand'] = 'other'

In [37]:
df_train = pd.get_dummies(df_train, columns=['carrier', 'devicebrand'], drop_first=True)
df_test = pd.get_dummies(df_test, columns=['carrier', 'devicebrand'], drop_first=True)

In [38]:
best_params = {
    "iterations": 1000,
    'subsample': 0.9389133536514731,
    'od_wait': 29,
    'learning_rate': 0.0066266955993544933,
    'depth': 5,
    'min_data_in_leaf': 70,
    'l2_leaf_reg': 22.95368957722943,
    'random_strength': 9,
    'max_bin': 255,
    'bootstrap_type': 'Bernoulli'  # "Bernoulli" olarak ayarlayın
}

In [40]:
X = df_train.drop(columns = ['id',"target"], axis=1)
y = df_train['target'].str.split(', ', expand=True)

In [41]:
X

,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,devicebrand_GM,devicebrand_HUAWEI,devicebrand_OPPO,devicebrand_POCO,devicebrand_Redmi,devicebrand_lge,devicebrand_other,devicebrand_realme,devicebrand_samsung,devicebrand_xiaomi
0,10,5245.571,981.182,205.948,-1.197737,1.113360,-1.123334,-0.263580,2.161242,2.651375,...,False,False,False,False,False,False,False,False,False,False
1,10,5184.876,557.650,487.587,-2.336352,2.567766,-0.494908,0.949101,3.567557,3.357848,...,False,False,False,False,False,False,False,False,True,False
2,10,3835.618,3275.128,43.806,-2.561455,2.061736,-0.184511,1.062306,4.197788,1.551181,...,False,False,False,False,True,False,False,False,False,False
3,10,3532.544,154.509,64.724,-2.529918,3.358050,-0.851366,1.643876,2.849205,3.887427,...,False,False,False,False,False,False,False,False,True,False
4,10,3344.192,787.896,715.115,-2.922361,2.096124,0.060796,-1.487557,3.224788,2.091947,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,12,44.397,43.425,41.678,-1.531534,2.596604,0.340233,-1.149720,3.308406,2.995904,...,False,True,False,False,False,False,False,False,False,False
94045,12,44.331,43.977,40.620,-1.268987,2.300487,0.231711,0.741582,2.365813,2.031927,...,False,False,False,False,False,False,False,False,True,False
94046,12,44.142,43.591,41.736,-1.950039,2.805681,0.438200,2.976427,4.001829,3.051181,...,False,False,False,False,False,False,False,False,False,False
94047,12,43.963,43.350,40.862,-2.389140,2.358281,0.683524,0.234449,4.070453,1.709853,...,False,False,False,False,True,False,False,False,False,False


In [42]:
y

,0,1,2
0,menu2,menu4,menu5
1,menu7,menu8,menu4
2,menu2,menu8,menu4
3,menu6,menu2,menu1
4,menu6,menu2,menu8
...,...,...,...
94044,menu2,menu8,menu4
94045,menu9,menu2,menu5
94046,menu6,menu2,menu4
94047,menu6,menu8,menu4


In [19]:
train_pool = Pool(X, y)
test_pool = Pool(df_test.drop(columns=['id'], axis=1))



In [20]:
clf = LabelPowerset(classifier = CatBoostClassifier(**best_params, silent=True, task_type="GPU", devices="0:1", loss_function="MultiClass", rsm=1))
clf.fit(pool, pool.get_label_matrix())

TypeError: no supported conversion for types: (dtype('O'),)

In [ ]:
y_pred = lp_classifier.predict(X_test)

In [14]:
def jaccard_similarity_score(y_true, y_pred):
    jaccard_scores = []
    for i in range(len(y_true)):
        intersection = np.sum(np.logical_and(y_true[i], y_pred[i]))
        union = np.sum(np.logical_or(y_true[i], y_pred[i]))
        jaccard_score = intersection / union
        jaccard_scores.append(jaccard_score)
    return np.mean(jaccard_scores)

In [16]:
#menu_combinations 

In [10]:
df_train

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,...,devicebrand_GM,devicebrand_HUAWEI,devicebrand_OPPO,devicebrand_POCO,devicebrand_Redmi,devicebrand_lge,devicebrand_other,devicebrand_realme,devicebrand_samsung,devicebrand_xiaomi
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,-1.197737,1.113360,-1.123334,-0.263580,2.161242,...,False,False,False,False,False,False,False,False,False,False
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,-2.336352,2.567766,-0.494908,0.949101,3.567557,...,False,False,False,False,False,False,False,False,True,False
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,-2.561455,2.061736,-0.184511,1.062306,4.197788,...,False,False,False,False,True,False,False,False,False,False
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,-2.529918,3.358050,-0.851366,1.643876,2.849205,...,False,False,False,False,False,False,False,False,True,False
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,-2.922361,2.096124,0.060796,-1.487557,3.224788,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,-1.531534,2.596604,0.340233,-1.149720,3.308406,...,False,True,False,False,False,False,False,False,False,False
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,-1.268987,2.300487,0.231711,0.741582,2.365813,...,False,False,False,False,False,False,False,False,True,False
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,-1.950039,2.805681,0.438200,2.976427,4.001829,...,False,False,False,False,False,False,False,False,False,False
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,-2.389140,2.358281,0.683524,0.234449,4.070453,...,False,False,False,False,True,False,False,False,False,False


In [14]:
X = df_train.drop(columns = ['id',"target"], axis=1)
y = df_train['target'] 

In [17]:
 # Elde ettiğiniz çoklu etiketli veri

lp = LabelPowerset()
y_train = lp.transform(y)

classifier = CatBoostClassifier(**best_params, silent=True, task_type="GPU", devices="0:1", loss_function="MultiClass", rsm=1)
train_data = Pool(X, y_train)
classifier.fit(train_data, use_best_model=True, verbose=100, early_stopping_rounds=50)

AttributeError: 'Series' object has no attribute 'reshape'

In [25]:
y_pred_lp = classifier.predict(Pool(df_test.drop(columns=['id'],axis=1)), prediction_type='Class')
y_pred_lp

array([[4],
       [3],
       [0],
       ...,
       [8],
       [8],
       [8]], dtype=int64)

In [29]:
y_pred_multiclass = lp.inverse_transform(y_pred_lp.ravel())
print(y_pred_multiclass )

  (0, 1)	1
  (0, 5)	1
  (0, 7)	1
  (1, 0)	1
  (1, 1)	1
  (1, 5)	1
  (2, 1)	1
  (2, 3)	1
  (2, 4)	1
  (3, 1)	1
  (3, 3)	1
  (3, 5)	1
  (4, 0)	1
  (4, 1)	1
  (4, 5)	1
  (5, 0)	1
  (5, 1)	1
  (5, 5)	1
  (6, 1)	1
  (6, 3)	1
  (6, 5)	1
  (7, 0)	1
  (7, 1)	1
  (7, 5)	1
  (8, 0)	1
  (8, 1)	1
  (8, 5)	1
  (9, 0)	1
  (9, 1)	1
  (9, 5)	1
  (10, 0)	1
  (10, 1)	1
  (10, 5)	1
  (11, 1)	1
  (11, 3)	1
  (11, 5)	1
  (12, 1)	1
  (12, 3)	1
  (12, 5)	1
  (13, 1)	1
  (13, 4)	1
  (13, 5)	1
  (14, 1)	1
  (14, 4)	1
  (14, 5)	1
  (15, 0)	1
  (15, 1)	1
  (15, 5)	1
  (16, 1)	1
  (16, 3)	1
  (16, 5)	1
  (17, 1)	1
  (17, 3)	1
  (17, 5)	1
  (18, 0)	1
  (18, 1)	1
  (18, 5)	1
  (19, 0)	1
  (19, 1)	1
  (19, 5)	1
  (20, 0)	1
  (20, 1)	1
  (20, 5)	1
  (21, 0)	1
  (21, 1)	1
  (21, 5)	1
  (22, 0)	1
  (22, 1)	1
  (22, 5)	1
  (23, 1)	1
  (23, 3)	1
  (23, 4)	1
  (24, 0)	1
  (24, 1)	1
  (24, 5)	1
  (25, 1)	1
  (25, 4)	1
  (25, 5)	1
  (26, 0)	1
  (26, 1)	1
  (26, 5)	1
  (27, 0)	1
  (27, 1)	1
  (27, 5)	1
  (28, 0)	1
  (28, 1)	

In [16]:
"""# Örnek veriler ve etiketler
X = df_train.drop(['id',"target"], axis=1)#,'month'
y = y_binary #df_train['target']  # Elde ettiğiniz çoklu etiketli veri
y_clean = y

kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
best_scores = {}

jaccard_scores_micro=[]
jaccard_scores_weighted=[]
jaccard_scores_macro=[]
jaccard_scores_func = []

for train_index, test_index in kf.split(X, y_clean):
    X_train, X_test =X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = [y_clean[i] for i in train_index], [y_clean[i] for i in test_index]
    
    lp = LabelPowerset()
    y_train = lp.transform(y_train)
    y_test = lp.transform(y_test)

    classifier = CatBoostClassifier(**best_params, silent=True, task_type="GPU", devices="0:1", loss_function="MultiClass", rsm=1)
    train_data = Pool(X_train, y_train)
    classifier.fit(train_data, use_best_model=True, verbose=100, early_stopping_rounds=50)
    
    y_pred = classifier.predict(Pool(X_test), prediction_type='Class')
    y_pred_labels = lp.inverse_transform(y_pred)
    
    for param,jlist in zip(['weighted', 'micro', 'macro'],[jaccard_scores_weighted,jaccard_scores_micro,jaccard_scores_macro]):
        score = jaccard_score(y_test, y_pred_labels,  average=param)  
        print(param, " jaccard_score=",score)
        jlist.append(score)   
    jaccard_scores_func.append(jaccard_similarity_score(y_test, y_pred))
    
print(f"micro = {np.mean(jaccard_scores_micro)}, weighted = {np.mean(jaccard_scores_weighted)}, macro = {np.mean(jaccard_scores_macro)}, func = {np.mean(jaccard_scores_func)}")"""

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

In [9]:
# Etiketleri virgülle ayır ve boşlukları kaldır
df_train['target'] = df_train['target'].apply(lambda x: [tag.strip() for tag in x.split(',')])

# MultiLabelBinarizer kullanarak etiketleri çoklu etiketli bir yapıya dönüştür
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(df_train['target'])
encoded_data = pd.DataFrame(encoded_labels, columns=mlb.classes_)
encoded_data

,menu1,menu2,menu3,menu4,menu5,menu6,menu7,menu8,menu9
0,0,1,0,1,1,0,0,0,0
1,0,0,0,1,0,0,1,1,0
2,0,1,0,1,0,0,0,1,0
3,1,1,0,0,0,1,0,0,0
4,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
94044,0,1,0,1,0,0,0,1,0
94045,0,1,0,0,1,0,0,0,1
94046,0,1,0,1,0,1,0,0,0
94047,0,0,0,1,0,1,0,1,0


In [59]:
# Örnek veriler ve etiketler
X = df_train.drop(['id',"target"], axis=1)#,'month'
y = encoded_data  # Elde ettiğiniz çoklu etiketli veri


kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
classifiers = []
best_scores = {}

for label in y.columns:
    jaccard_scores_micro=[]
    jaccard_scores_weighted=[]
    jaccard_scores_macro=[]
    
    y_temp = y[label]
    for train_index, test_index in kf.split(X, y_temp):
        #print(train_index,test_index)
        X_train, X_test =X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = [y_temp[i] for i in train_index], [y_temp[i] for i in test_index]
        
        classifier = CatBoostClassifier(**best_params, silent=True, task_type="GPU", devices="0:1", loss_function="MultiClass", rsm=1)
        classifier.fit(X_train, y_train, use_best_model=True, verbose=100, early_stopping_rounds=50)
        
        #model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True, verbose=100, early_stopping_rounds=50)
        y_pred = classifier.predict(X_test, prediction_type='Class')

        #y_pred = encoder.inverse_transform(y_pred)
        for param,jlist in zip(['weighted', 'micro', 'macro'],[jaccard_scores_weighted,jaccard_scores_micro,jaccard_scores_macro]):
            score = jaccard_score(y_test, y_pred,  average=param)  
            print(param, " jaccard_score=",score)
            jlist.append(score)   

    print(f"micro = {np.mean(jaccard_scores_micro)}, weighted = {np.mean(jaccard_scores_weighted)}, macro = {np.mean(jaccard_scores_macro)}")
    best_scores[label] = np.mean(jaccard_scores_micro)
    #classifiers.append(classifier)
    #df_test[label] = classifier.predict(X_test)
    
#final_predictions = X_test[y.columns]

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6912242	total: 64.7ms	remaining: 1m 4s
100:	learn: 0.5816933	total: 1.33s	remaining: 11.8s
200:	learn: 0.5469983	total: 2.59s	remaining: 10.3s
300:	learn: 0.5349402	total: 3.76s	remaining: 8.73s
400:	learn: 0.5311448	total: 4.44s	remaining: 6.63s
500:	learn: 0.5299823	total: 4.91s	remaining: 4.89s
600:	learn: 0.5295899	total: 5.27s	remaining: 3.5s
700:	learn: 0.5293420	total: 5.61s	remaining: 2.39s
800:	learn: 0.5291507	total: 5.95s	remaining: 1.48s
900:	learn: 0.5289719	total: 6.31s	remaining: 694ms
999:	learn: 0.5288363	total: 6.65s	remaining: 0us
weighted  jaccard_score= 0.581514162073803
micro  jaccard_score= 0.6160692807312966
macro  jaccard_score= 0.38197345223994733


You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6912312	total: 24.6ms	remaining: 24.6s
100:	learn: 0.5818743	total: 1.13s	remaining: 10s
200:	learn: 0.5472702	total: 2.03s	remaining: 8.08s
300:	learn: 0.5351857	total: 3s	remaining: 6.97s
400:	learn: 0.5314171	total: 3.6s	remaining: 5.38s
500:	learn: 0.5302662	total: 4.12s	remaining: 4.11s
600:	learn: 0.5298747	total: 4.53s	remaining: 3s
700:	learn: 0.5295499	total: 4.93s	remaining: 2.1s
800:	learn: 0.5293299	total: 5.28s	remaining: 1.31s
900:	learn: 0.5291951	total: 5.6s	remaining: 616ms
999:	learn: 0.5290656	total: 5.98s	remaining: 0us
weighted  jaccard_score= 0.5815591423476014
micro  jaccard_score= 0.6162227186801856
macro  jaccard_score= 0.38186415252873157


You should provide test set for use best model. use_best_model parameter has been switched to false value.


...

0:	learn: 0.6914512	total: 19ms	remaining: 19s
100:	learn: 0.5943741	total: 1.52s	remaining: 13.5s
200:	learn: 0.5638907	total: 2.96s	remaining: 11.8s
300:	learn: 0.5535341	total: 4.25s	remaining: 9.87s
400:	learn: 0.5502056	total: 5.12s	remaining: 7.65s
500:	learn: 0.5492787	total: 5.64s	remaining: 5.62s
600:	learn: 0.5489282	total: 6.13s	remaining: 4.07s
700:	learn: 0.5487682	total: 6.55s	remaining: 2.79s
800:	learn: 0.5485965	total: 6.97s	remaining: 1.73s
900:	learn: 0.5484747	total: 7.36s	remaining: 809ms
999:	learn: 0.5483724	total: 7.75s	remaining: 0us
weighted  jaccard_score= 0.5639044141388683
micro  jaccard_score= 0.6011985834922364
macro  jaccard_score= 0.37546784620619256


You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6914462	total: 16.3ms	remaining: 16.3s
100:	learn: 0.5944305	total: 1.37s	remaining: 12.2s
200:	learn: 0.5640709	total: 2.73s	remaining: 10.8s
300:	learn: 0.5538079	total: 4.13s	remaining: 9.59s
400:	learn: 0.5506555	total: 4.88s	remaining: 7.29s
500:	learn: 0.5496908	total: 5.36s	remaining: 5.34s
600:	learn: 0.5493451	total: 5.8s	remaining: 3.85s
700:	learn: 0.5491618	total: 6.23s	remaining: 2.66s
800:	learn: 0.5490156	total: 6.69s	remaining: 1.66s
900:	learn: 0.5489050	total: 7.13s	remaining: 784ms
999:	learn: 0.5488250	total: 7.54s	remaining: 0us
weighted  jaccard_score= 0.5639044141388683
micro  jaccard_score= 0.6011985834922364
macro  jaccard_score= 0.37546784620619256


You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6914506	total: 17ms	remaining: 17s
100:	learn: 0.5941306	total: 1.42s	remaining: 12.6s
200:	learn: 0.5635876	total: 2.87s	remaining: 11.4s
300:	learn: 0.5531388	total: 4.12s	remaining: 9.56s
400:	learn: 0.5500276	total: 4.77s	remaining: 7.13s
500:	learn: 0.5491662	total: 5.22s	remaining: 5.2s
600:	learn: 0.5487726	total: 5.67s	remaining: 3.76s
700:	learn: 0.5486057	total: 6.08s	remaining: 2.59s
800:	learn: 0.5484307	total: 6.57s	remaining: 1.63s
900:	learn: 0.5483027	total: 7s	remaining: 769ms
999:	learn: 0.5481516	total: 7.47s	remaining: 0us
weighted  jaccard_score= 0.5638405391451056
micro  jaccard_score= 0.6011440634682829
macro  jaccard_score= 0.3754465804695475
micro = 0.6011747185767572, weighted = 0.5638764543126662, macro = 0.37545853764165443


In [60]:
print(best_scores)
sorted_scores = dict(sorted(best_scores.items(), key=lambda item: item[1], reverse=True))
print(sorted_scores)

{'menu1': 0.6161843677846218, 'menu2': 0.6574555457899375, 'menu3': 0.8866777679263311, 'menu4': 0.451016909601737, 'menu5': 0.6754226098903706, 'menu6': 0.5279849232112883, 'menu7': 0.9158289281778362, 'menu8': 0.6035538762519248, 'menu9': 0.6011747185767572}
{'menu7': 0.9158289281778362, 'menu3': 0.8866777679263311, 'menu5': 0.6754226098903706, 'menu2': 0.6574555457899375, 'menu1': 0.6161843677846218, 'menu8': 0.6035538762519248, 'menu9': 0.6011747185767572, 'menu6': 0.5279849232112883, 'menu4': 0.451016909601737}


In [63]:
for label in sorted_scores.keys():
    print(label)
    
    classifier = CatBoostClassifier(**best_params, silent=True, task_type="GPU", devices="0:1", loss_function="MultiClass", rsm=1)
    classifier.fit(X, y[label], use_best_model=True, verbose=100, early_stopping_rounds=50)
    y_pred = classifier.predict(df_test.drop(columns=['id'],axis=1), prediction_type='Class')
    X[label] = y[label]
    print(X.columns)
    df_test[label] = y_pred

menu7


You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6877185	total: 17.9ms	remaining: 17.9s
100:	learn: 0.3681205	total: 1.61s	remaining: 14.3s
200:	learn: 0.2508179	total: 3.07s	remaining: 12.2s
300:	learn: 0.2038287	total: 4.53s	remaining: 10.5s
400:	learn: 0.1851769	total: 5.95s	remaining: 8.89s
500:	learn: 0.1779793	total: 7.09s	remaining: 7.07s
600:	learn: 0.1755493	total: 7.75s	remaining: 5.15s
700:	learn: 0.1747713	total: 8.29s	remaining: 3.54s
800:	learn: 0.1744771	total: 8.82s	remaining: 2.19s
900:	learn: 0.1743544	total: 9.31s	remaining: 1.02s
999:	learn: 0.1742957	total: 9.75s	remaining: 0us
Index(['month', 'n_seconds_1', 'n_seconds_2', 'n_seconds_3', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6880770	total: 18.6ms	remaining: 18.6s
100:	learn: 0.3919928	total: 1.49s	remaining: 13.3s
200:	learn: 0.2862140	total: 2.91s	remaining: 11.6s
300:	learn: 0.2453986	total: 4.27s	remaining: 9.91s
400:	learn: 0.2296914	total: 5.83s	remaining: 8.7s
500:	learn: 0.2238528	total: 6.92s	remaining: 6.89s
600:	learn: 0.2220562	total: 7.56s	remaining: 5.02s
700:	learn: 0.2214442	total: 8.15s	remaining: 3.48s
800:	learn: 0.2212111	total: 8.72s	remaining: 2.17s
900:	learn: 0.2211109	total: 9.19s	remaining: 1.01s
999:	learn: 0.2209777	total: 9.72s	remaining: 0us
Index(['month', 'n_seconds_1', 'n_seconds_2', 'n_seconds_3', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_2

...

0:	learn: 0.6904672	total: 17.9ms	remaining: 17.9s
100:	learn: 0.5340695	total: 1.59s	remaining: 14.2s
200:	learn: 0.4791851	total: 3.23s	remaining: 12.8s
300:	learn: 0.4585806	total: 4.83s	remaining: 11.2s
400:	learn: 0.4501087	total: 6.07s	remaining: 9.06s
500:	learn: 0.4470304	total: 7.23s	remaining: 7.2s
600:	learn: 0.4457593	total: 7.94s	remaining: 5.27s
700:	learn: 0.4448903	total: 8.56s	remaining: 3.65s
800:	learn: 0.4442299	total: 9.27s	remaining: 2.3s
900:	learn: 0.4436147	total: 9.84s	remaining: 1.08s
999:	learn: 0.4430111	total: 10.4s	remaining: 0us
Index(['month', 'n_seconds_1', 'n_seconds_2', 'n_seconds_3', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6906067	total: 22.1ms	remaining: 22.1s
100:	learn: 0.5409690	total: 1.58s	remaining: 14s
200:	learn: 0.4831522	total: 3s	remaining: 11.9s
300:	learn: 0.4523569	total: 4.31s	remaining: 10s
400:	learn: 0.4360976	total: 5.59s	remaining: 8.35s
500:	learn: 0.4273008	total: 6.58s	remaining: 6.55s
600:	learn: 0.4214894	total: 7.49s	remaining: 4.97s
700:	learn: 0.4169047	total: 8.32s	remaining: 3.55s
800:	learn: 0.4130948	total: 9.15s	remaining: 2.27s
900:	learn: 0.4098929	total: 9.93s	remaining: 1.09s
999:	learn: 0.4071174	total: 10.7s	remaining: 0us
Index(['month', 'n_seconds_1', 'n_seconds_2', 'n_seconds_3', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'f

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 0.6896764	total: 19ms	remaining: 19s
100:	learn: 0.4722082	total: 1.6s	remaining: 14.2s
200:	learn: 0.3745300	total: 3.36s	remaining: 13.4s
300:	learn: 0.3034595	total: 4.81s	remaining: 11.2s
400:	learn: 0.2489605	total: 6.17s	remaining: 9.22s
500:	learn: 0.2068367	total: 7.59s	remaining: 7.56s
600:	learn: 0.1729840	total: 9.05s	remaining: 6.01s
700:	learn: 0.1466275	total: 10.5s	remaining: 4.49s
800:	learn: 0.1260784	total: 11.9s	remaining: 2.95s
900:	learn: 0.1094361	total: 13.1s	remaining: 1.44s
999:	learn: 0.0977940	total: 14.3s	remaining: 0us
Index(['month', 'n_seconds_1', 'n_seconds_2', 'n_seconds_3', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 

In [64]:
df_test[['menu1','menu2','menu3','menu4','menu5','menu6','menu7','menu8','menu9']]

,menu1,menu2,menu3,menu4,menu5,menu6,menu7,menu8,menu9
0,0,1,0,1,0,1,0,0,0
1,0,1,0,1,0,1,0,0,0
2,0,1,0,0,0,1,0,1,0
3,0,1,0,1,0,1,0,0,0
4,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
11950,0,1,0,1,0,1,0,0,0
11951,0,1,0,1,0,1,0,0,0
11952,0,1,0,1,0,1,0,0,0
11953,0,1,0,1,0,1,0,0,0


In [68]:
pf_sub = ParquetFile("submission_sample_final.parquet")
df_sub = pf_sub.to_pandas()
df_sub.target = df_test[['menu1','menu2','menu3','menu4','menu5','menu6','menu7','menu8','menu9']].astype(str).agg(''.join, axis=1)
df_sub.to_csv('df_binary.csv',index=False)

In [71]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [75]:
numerical_columns = df_train.select_dtypes(exclude=['object']).columns.tolist()
for label in sorted_scores.keys():
    correlations = {}
    for feature_column in numerical_columns:
        correlation = X[label].corr(df_train[feature_column])
        correlations[feature_column] = correlation
        sorted_correlations = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

    # En yüksek korelasyona sahip sütunları gösterin
    top_correlations = sorted_correlations[:4]  # İlk 10 sütunu alabilirsiniz
    print("Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:")
    for column, correlation in top_correlations:
        print(f"{column}: {correlation}")

Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:
corr_48_49: -0.06693198637789642
feature_49: -0.06509715674312698
month: -0.05245180035581203
feature_48: -0.05219106597202664
Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:
feature_29: -0.041582386907880096
feature_9: 0.040961128703639535
feature_21: -0.039205082120058445
feature_32: 0.03791905269413524
Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:
feature_26: -0.24865903929880634
feature_11: 0.23129586899084528
feature_21: 0.18848652360336965
feature_25: -0.18573901688966646
Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:
n_seconds_1: 0.08998658584351327
n_seconds_2: 0.08825042997349308
feature_26: 0.0735143816951973
feature_31: -0.06962518704072561
Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:
feature_49: 0.10260815706251865
feature_25: 0.10240132236575834
corr_48_49: 0.08229178210780648
feature_16: 0.07728914668819642
Kategorik Hedef ile En Yüksek Korelasyona Sahip Sütunlar:
n_sec

In [77]:
from itertools import product

In [78]:
all_label_combinations = list(product([0, 1], repeat=len(y.columns)))
all_label_combinations

[(0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 1, 1),
 (0, 0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 0, 1, 0, 1, 0),
 (0, 0, 0, 0, 0, 1, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 1, 0, 0),
 (0, 0, 0, 0, 0, 1, 1, 0, 1),
 (0, 0, 0, 0, 0, 1, 1, 1, 0),
 (0, 0, 0, 0, 0, 1, 1, 1, 1),
 (0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 0, 1, 1),
 (0, 0, 0, 0, 1, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 0, 1, 0, 1),
 (0, 0, 0, 0, 1, 0, 1, 1, 0),
 (0, 0, 0, 0, 1, 0, 1, 1, 1),
 (0, 0, 0, 0, 1, 1, 0, 0, 0),
 (0, 0, 0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 1, 0, 1, 0),
 (0, 0, 0, 0, 1, 1, 0, 1, 1),
 (0, 0, 0, 0, 1, 1, 1, 0, 0),
 (0, 0, 0, 0, 1, 1, 1, 0, 1),
 (0, 0, 0, 0, 1, 1, 1, 1, 0),
 (0, 0, 0, 0, 1, 1, 1, 1, 1),
 (0, 0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 0,